# <B> # SageMaker monitor for data and model quality </B>
* Container: codna_python3
    - Model Quaility
        - https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/model_quality/model_quality_churn_sdk.html
        - https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker_model_monitor/model_quality/model_quality_churn_sdk.ipynb
    - Data Quaility
        - https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker_model_monitor/introduction
        - https://github.com/aws-samples/amazon-sagemaker-data-quality-monitor-custom-preprocessing
        - https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/introduction/SageMaker-ModelMonitoring.html#Create-a-baselining-job-with-training-dataset
    - 컬럼수 안맞을때
        - https://repost.aws/questions/QU8Xkelo1ARA2zcn4rHuk09w/questions/QU8Xkelo1ARA2zcn4rHuk09w/sagemaker-model-monitor-missing-columns-constraint-violation?

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [2]:
install_needed = False  # should only be True once
# install_needed = False

In [3]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U xgboost==1.3.1

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached pip-23.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.147.0-py2.py3-none-any.whl size=965137 sha256=67f4809e70249bcb69cd12d6cceeb8d28ecc938a14f4a495c847397c35c2f880
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a7/82/1f/1479d5a2d69b3429533baa517af5a66041c69cb3e2ca5d1041
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.146.0
  

## 1. parameter store 설정

In [3]:
import boto3
from utils.ssm import parameter_store

In [4]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

In [5]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))
strExecutionRole = pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))

In [6]:
print (f'strBucketName: {strBucketName}')
print (f'strExecutionRole: {strExecutionRole}')

strBucketName: sagemaker-us-east-1-419974056037
strExecutionRole: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436


## 2. Dataset

In [7]:
import os

In [8]:
strS3DataPath = f"s3://{strBucketName}/dataset" 
strLocalDataPath = os.path.join(os.getcwd(), "data")

## 3.Depoly with Data capture
- https://github.com/aws-samples/amazon-sagemaker-data-quality-monitor-custom-preprocessing
- https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker_model_monitor/introduction
- https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/introduction/SageMaker-ModelMonitoring.html#Create-a-baselining-job-with-training-dataset

### 3.1 Check functions in local mode
[중요] inference.py를 만들어 주어야 함
* model_fn: 학습한 모델 로드
* input_fn: endpoint invocation시 전달 되는 input 처리 하는 함수
* predict_fn: forword propagation, input_fn의 이후 호출 
* output_fn: 유저에게 결과 전달

- 사용자 정의 inference 코드를 정의해서 사용하기 전에, 노트북에서 사전 테스트 및 디버깅을 하고 진행하면 빠르게 추론 개발을 할수 있습니다.
- 디폴트 inference code (input_fn, model_fn, predict_fn, output_fn) 을 사용해도 되지만, 상황에 따라서는 사용자 정의가 필요할 수 있습니다. 디폴트 코드는 아래 링크를 참고 하세요.
    - [Deploy PyTorch Models](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#deploy-pytorch-models)
    - [디폴트 inference Code](https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/master/src/sagemaker_pytorch_serving_container/default_pytorch_inference_handler.py)

### 로컬 모드 수행시, 새로운 로컬모드 수행을 위해서는 이전 사용했던 도커는 반드시 stop 해줘야 한다
* docker ps -a 로 현재 수행중인 contatiner ID 확인 후
* docker stop "<<contatiner ID>>"
* docker container prune -f

* 3.1.1 inference.py 생성
    - https://aws.amazon.com/ko/blogs/machine-learning/design-a-compelling-record-filtering-method-with-amazon-sagemaker-model-monitor/
    -  We also need to ensure that Flask Response is returned to match both input and output content types exactly. It is a necessary step for Model Monitor to work for the image running Gunicorn/Flask. The content type of output data captured by Model Monitor, which only works with CSV or JSON, is Base64 by default unless Response() explicitly converts it to a specific type.

In [10]:
%%writefile source/deploy/inference.py
import io
import os
import csv
import time
import json
import pickle as pkl
import numpy as np
import pandas as pd
from io import BytesIO
import xgboost as xgb
import sagemaker_xgboost_container.encoder as xgb_encoders
from sagemaker.serializers import CSVSerializer
from io import StringIO

#For Gunicorn/Flask xgboost image, we need to ensure input and output encoding match exactly for model monitor (CSV or JSON)
from flask import Response 

NUM_FEATURES = 58
CSV_SERIALIZER = CSVSerializer(content_type='text/csv')

def model_fn(model_dir):
    """
    Deserialize and return fitted model.
    """
    model_file = "xgboost-model"
    model = xgb.Booster()
    model.load_model(os.path.join(model_dir, model_file))
    return model
                     

def input_fn(request_body, request_content_type):
    """
    The SageMaker XGBoost model server receives the request data body and the content type,
    and invokes the `input_fn`.
    Return a DMatrix (an object that can be passed to predict_fn).
    """

    print (f'Input, Content_type: {request_content_type}')
    if request_content_type == "application/x-npy":        
        stream = BytesIO(request_body)
        array = np.frombuffer(stream.getvalue())
        array = array.reshape(int(len(array)/NUM_FEATURES), NUM_FEATURES)
        return xgb.DMatrix(array)
    
    elif request_content_type == "text/csv":
        return xgb_encoders.csv_to_dmatrix(request_body.rstrip("\n"))
    
    elif request_content_type == "text/libsvm":
        return xgb_encoders.libsvm_to_dmatrix(request_body)
    
    else:
        raise ValueError(
            "Content type {} is not supported.".format(request_content_type)
        )

def predict_fn(input_data, model):
    """
    SageMaker XGBoost model server invokes `predict_fn` on the return value of `input_fn`.

    Return a two-dimensional NumPy array (predictions and scores)
    """
    start_time = time.time()
    y_probs = model.predict(input_data)
    print("--- Inference time: %s secs ---" % (time.time() - start_time))    
    y_preds = [1 if e >= 0.5 else 0 for e in y_probs] 
    #return np.vstack((y_preds, y_probs))
    y_probs = np.array(y_probs).reshape(1, -1)
    y_preds = np.array(y_preds).reshape(1, -1)   
    output = np.concatenate([y_probs, y_preds], axis=1)
    
    return output


def output_fn(predictions, content_type="text/csv"):
    """
    After invoking predict_fn, the model server invokes `output_fn`.
    """
    print (f'Output, Content_type: {content_type}')
    
    if content_type == "text/csv":
        outputs = CSV_SERIALIZER.serialize(predictions)
        print (outputs)
        return Response(outputs, mimetype=content_type)

    elif content_type == "application/json":

        outputs = json.dumps({
            'pred': predictions[0][0],
            'prob': predictions[0][1]
        })                
        #return outputs
        return Response(outputs, mimetype=content_type)
    else:
        raise ValueError("Content type {} is not supported.".format(content_type))

Overwriting source/deploy/inference.py


* 3.1.2 param setting

In [11]:
import time
import sagemaker
from sagemaker.model_monitor import DataCaptureConfig

In [12]:
local_mode = False
strEndpointName = "endpoint-cloud-DJ-SM-IMD-1682062443" # None

if local_mode:
    
    from sagemaker.local import LocalSession
    
    strInstanceType = "local"
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    strDeployType = "local"
        
else:
    strInstanceType = "ml.m5.xlarge"
    sagemaker_session = sagemaker.Session()
    strDeployType = "cloud"
    
strS3ModelPath = pm.get_params(key="-".join([strPrefix, "MODEL-PATH"]))
if strEndpointName == None:
    strEndpointName = f"endpoint-{strDeployType}-{strPrefix}-{int(time.time())}" 

strS3DataCapturePath = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "monitor",
    "data-capture"
)

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=strS3DataCapturePath,
    capture_options=["REQUEST", "RESPONSE"],
    csv_content_types=["text/csv"]
)

In [13]:
print (f'strInstanceType: {strInstanceType}')
print (f'sagemaker_session: {sagemaker_session}')
print (f'strS3ModelPath: {strS3ModelPath}')
print (f'strEndpointName: {strEndpointName}')

strInstanceType: ml.m5.xlarge
sagemaker_session: <sagemaker.session.Session object at 0x7fd92d08f8e0>
strS3ModelPath: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/training/model-output/DJ-SM-IMD-experiments-0419-04191681877971/output/model.tar.gz
strEndpointName: endpoint-cloud-DJ-SM-IMD-1682062443


* Create model

In [19]:
from sagemaker.xgboost.model import XGBoostModel
from sagemaker.serializers import CSVSerializer, NumpySerializer, JSONSerializer
from sagemaker.deserializers import CSVDeserializer, JSONDeserializer, NumpyDeserializer

In [120]:
xgb_model = XGBoostModel(
    model_data=strS3ModelPath,
    role=strExecutionRole,
    source_dir="./source/deploy",
    entry_point="inference.py",
    framework_version="1.3-1",
    sagemaker_session=sagemaker_session,
)

* Create Endpoint with **data capture**
    * SageMaker SDK는 `deploy(...)` 메소드를 호출 시, `create-endpoint-config`와 `create-endpoint`를 같이 수행합니다. 좀 더 세분화된 파라메터 조정을 원하면 AWS CLI나 boto3 SDK client 활용을 권장 드립니다.

In [104]:
xgb_predictor = xgb_model.deploy(
    endpoint_name=strEndpointName,
    instance_type=strInstanceType, 
    initial_instance_count=1,
    data_capture_config=data_capture_config,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
    wait=True,
    log=True,
)

-nj5in58f4q-algo-1-x7hac | [2023-04-21 01:27:25 +0000] [73] [INFO] Handling signal: term
nj5in58f4q-algo-1-x7hac | [2023-04-21 01:27:25 +0000] [79] [INFO] Worker exiting (pid: 79)
nj5in58f4q-algo-1-x7hac | [2023-04-21 01:27:25 +0000] [104] [INFO] Worker exiting (pid: 104)
nj5in58f4q-algo-1-x7hac | [2023-04-21 01:27:25 +0000] [87] [INFO] Worker exiting (pid: 87)
nj5in58f4q-algo-1-x7hac | [2023-04-21 01:27:25 +0000] [88] [INFO] Worker exiting (pid: 88)
nj5in58f4q-algo-1-x7hac exited with code 0
Aborting on container exit...
-----!

* inference (based on **boto3**)
    - **boto3 기반 invocation시 runtime_client가 필요**
    - deploy 시 설정했던 "serialization, deserialization"이 적용되지 않음, 즉, **serialization, deserialization을 manually 해 줘야 함**
        - 번거로울 수 있으나 de/serialization에 대한 자유도가 높음

In [20]:
import json
import boto3
import sagemaker
import pandas as pd
import numpy as np

In [21]:
if "local" in strInstanceType: runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
else: runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7fd92be4f010>


In [22]:
pdTest = pd.read_csv(f'{strLocalDataPath}/test.csv')
pdTest = pdTest.drop('fraud', axis=1)

* serialzaiton (csv)

In [23]:
csv_serializer = CSVSerializer()
csv_deserializer = CSVDeserializer()

In [24]:
payload = csv_serializer.serialize(pdTest.values[122, :])

In [25]:
payload,strEndpointName

('7738.118217725296,11938.118217725296,39.0,61.0,0.0,1.0,750.0,3000.0,95815.0,2015.0,2.0,0.0,1.0,4200.0,12.0,6.0,4.0,22.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0',
 'endpoint-cloud-DJ-SM-IMD-1682062443')

In [26]:
response = runtime_client.invoke_endpoint(
    EndpointName=strEndpointName, 
    ContentType='text/csv',
    Accept='text/csv',
    Body=payload
)
pred = np.array(
    csv_deserializer.deserialize(
        stream=response['Body'],
        content_type="text/csv"
    ),
    dtype=np.float32
)
pred

array([[0.20726663, 0.        ]], dtype=float32)

## 4. View captured data

In [27]:
import json

In [28]:
s3_client = boto3.Session().client("s3")

In [29]:
def get_obj_body(obj_key, strBucketName):
    return s3_client.get_object(Bucket=strBucketName, Key=obj_key).get("Body").read().decode("utf-8")

In [30]:
current_endpoint_capture_prefix = os.path.join(
    strPrefix,
    "monitor",
    "data-capture",
    strEndpointName
)
result = s3_client.list_objects(Bucket=strBucketName, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))
print (capture_files[len(capture_files) - 1][:capture_files[len(capture_files) - 1].rfind("/")])

Found Capture Files:
DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/37-45-420-13675920-9041-43dc-ba75-30351ca76e99.jsonl
 DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/40-14-247-5cb76178-c578-4215-8f08-33e9b164eb8f.jsonl
DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07


In [31]:
capture_file = get_obj_body(capture_files[-1], strBucketName)
#print(capture_file.split("\n")[-2])
print(json.dumps(json.loads(capture_file.split("\n")[-2]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "17047.719421914477,28347.719421914484,52.0,51.0,0.0,1.0,750.0,2650.0,94601.0,2020.0,1.0,0.0,0.0,11300.0,1.0,21.0,0.0,11.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.15399029850959778,0.0",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "28ee34cc-db62-4222-ac13-a7993441f28c",
    "inferenceTime": "2023-04-21T07:40:29Z"
  },
  "eventVersion": "0"
}


## 5. Model Monitor - Baselining and continuous monitoring

### 5.1 Execute predictions using the test dataset.

In [32]:
from time import sleep

In [33]:
pdTest = pd.read_csv(f'{strLocalDataPath}/test.csv')

* **[중요]** BinaryClassification에서 lable 및 prediction은 **integer** 형태여야 함

* Notice the new attribute `inferenceId`, which we're setting when invoking the endpoint. This is used to join the prediction data with the ground truth data.
    - You can check this in jsonl file (data captured)
    - inferenceId attribute that is set as part of the invoke_endpoint call. If this is present, it will be used to join with ground truth data (otherwise `eventId` will be used)

In [34]:
%%time

if "local" in strInstanceType: runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
else: runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')


csv_serializer = CSVSerializer()
csv_deserializer = CSVDeserializer()
nLimit = 200

with open(f'{strLocalDataPath}/model_baseline_data.csv', "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # header
    for idx, row in enumerate(pdTest.values):    
        nLabel, fFeatures = int(row[0]), row[1:]
        payload = csv_serializer.serialize(fFeatures)
        response = runtime_client.invoke_endpoint(
            EndpointName=strEndpointName, 
            ContentType='text/csv',
            Accept='text/csv',
            Body=payload,
            InferenceId=str(idx)
        )

        predictions = np.array(
            csv_deserializer.deserialize(
                stream=response['Body'],
                content_type="text/csv"
            ),
            dtype=np.float32
        ).flatten()
        fProb, nPred = predictions[0], int(predictions[1])
        #print (idx, fProb, nPred)
        if idx % 10 == 0: print(idx, end="", flush=True)
        if idx >= nLimit: break
        baseline_file.write(f"{fProb},{nPred},{nLabel}\n")
        #print(".", end="", flush=True)
        sleep(0.5)


runtime_client: <botocore.client.SageMakerRuntime object at 0x7fd92b020850>
0 0.13204898 0
01 0.055153064 0
2 0.5491993 1
3 0.1765169 0
4 0.5688996 1
5 0.29007915 0
6 0.5657198 1
7 0.5151643 1
8 0.7608245 1
9 0.1539903 0
10 0.56986153 1
1011 0.6299357 1
12 0.022774024 0
13 0.1539903 0
14 0.6334661 1
15 0.12232828 0
16 0.29007915 0
17 0.20726663 0
18 0.1539903 0
19 0.08634089 0
20 0.4788133 0
2021 0.12121122 0
22 0.13121638 0
23 0.28858143 0
24 0.29007915 0
25 0.5542213 1
26 0.054573808 0
27 0.1539903 0
28 0.12232828 0
29 0.71389216 1
30 0.1539903 0
3031 0.15050104 0
32 0.12232828 0
33 0.22885583 0
34 0.13088676 0
35 0.5196692 1
36 0.19419017 0
37 0.5239408 1
38 0.20070212 0
39 0.5607354 1
40 0.055153064 0
4041 0.6337742 1
42 0.31524706 0
43 0.6320183 1
44 0.4788133 0
45 0.10985424 0
46 0.4788133 0
47 0.029054698 0
48 0.033639323 0
49 0.1539903 0
50 0.05553387 0
5051 0.32956877 0
52 0.71389216 1
53 0.20726663 0
54 0.05553387 0
55 0.1721504 0
56 0.20726663 0
57 0.71389216 1
58 0.4788133 

### 5.2 Examine the predictions from the model

In [35]:
!head $f'{strLocalDataPath}/model_baseline_data.csv'

probability,prediction,label
0.13204897940158844,0,0
0.05515306442975998,0,0
0.5491992831230164,1,0
0.17651690542697906,0,0
0.5688995718955994,1,0
0.29007914662361145,0,0
0.5657197833061218,1,0
0.515164315700531,1,0
0.7608245015144348,1,0


### 5.3 Upload the predictions as a baseline dataset.

In [36]:
strS3ModelBaselinePrefix = os.path.join(
    strPrefix,
    "monitor",
    "baselining",
    "model"
)
strS3ModelBaselineDataPrefix = os.path.join(
    strS3ModelBaselinePrefix,
    "data"
)
strS3ModelBaselineResultsPrefix = os.path.join(
    strS3ModelBaselinePrefix,
    "results"
)
strS3ModelBaselineDataUri = os.path.join(
    "s3://{}".format(strBucketName),
    strS3ModelBaselineDataPrefix
)
strS3ModelBaselineResultsUri = os.path.join(
    "s3://{}".format(strBucketName),
    strS3ModelBaselineResultsPrefix
)

print (f'strS3ModelBaselinePrefix: {strS3ModelBaselinePrefix}')
print (f'strS3ModelBaselineDataUri: {strS3ModelBaselineDataUri}')
print (f'strS3ModelBaselineResultsUri: {strS3ModelBaselineResultsUri}')

strS3ModelBaselinePrefix: DJ-SM-IMD/monitor/baselining/model
strS3ModelBaselineDataUri: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/baselining/model/data
strS3ModelBaselineResultsUri: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/baselining/model/results


In [37]:
model_baseline_data_file = open(f'{strLocalDataPath}/model_baseline_data.csv', "rb")
s3_key = os.path.join(strS3ModelBaselinePrefix, "data", "model_baseline_data.csv")
boto3.Session().resource("s3").Bucket(strBucketName).Object(s3_key).upload_fileobj(model_baseline_data_file)

### 5.4 Create a baselining job with test dataset predictions

In [38]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

In [39]:
# Create the model quality monitoring object
model_quality_monitor = ModelQualityMonitor(
    role=strExecutionRole,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)

In [40]:
# Execute the baseline suggestion job.
# You will specify problem type, in this case Binary Classification, and provide other required attributes.
model_quality_monitor.suggest_baseline(
    #job_name=baseline_job_name,
    baseline_dataset=os.path.join(
        strS3ModelBaselineDataUri,
        "model_baseline_data.csv"
    ),
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=strS3ModelBaselineResultsUri,
    problem_type="BinaryClassification",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
    wait=True
)

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2023-04-21-07-49-52-209


...............................................................................!

### 5.5 Explore the results of the baselining job

In [41]:
model_baseline_job = model_quality_monitor.latest_baselining_job

* View the metrics generated

In [42]:
binary_metrics = model_baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T

,0
confusion_matrix.0.0,156
confusion_matrix.0.1,38
confusion_matrix.1.0,2
confusion_matrix.1.1,4
recall.value,0.666667
recall.standard_deviation,0.149666
precision.value,0.095238
precision.standard_deviation,0.009601
accuracy.value,0.8
accuracy.standard_deviation,0.015993


* View the constraints generated
    - 베이스라인 만드는 데이테 셋에 대한 performance가 threshod가 된다.

In [43]:
pd.DataFrame(model_baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.666667,LessThanThreshold
precision,0.095238,LessThanThreshold
accuracy,0.8,LessThanThreshold
true_positive_rate,0.666667,LessThanThreshold
true_negative_rate,0.804124,LessThanThreshold
false_positive_rate,0.195876,GreaterThanThreshold
false_negative_rate,0.333333,GreaterThanThreshold
auc,0.856529,LessThanThreshold
f0_5,0.114943,LessThanThreshold
f1,0.166667,LessThanThreshold


## 6. Model Monitor - Setup continuous model monitoring to identify model quality drift

In this section, you will setup a continuous model monitoring job that `monitors the quality of the deployed model` against the baseline generated in the previous section. This is to ensure that the quality does not degrade over time.

In addition to the generated baseline, Amazon SageMaker Model Quality Monitoring `needs two additional inputs` - `predictions made by the deployed model endpoint` and the `ground truth data to be provided by the model` consuming application. Since you already enabled data capture on the endpoint, prediction data is captured in S3. The ground truth data depends on the what your model is predicting and what the business use case is. In this case, since the model is predicting customer churn, ground truth data may indicate if the customer actually left the company or not. For the purposes of this notebook, you will generate synthetic data as ground truth.

### 6.1 Generate prediction data for Model Quality Monitoring
    - invocation 시 inferenceid를 전달함으로써 inference를 한 대상이 무엇인지 기록한다. 

In [44]:
from threading import Thread

In [45]:
pdTest = pd.read_csv(f'{strLocalDataPath}/test.csv')

In [46]:
def invoke_endpoint(strEndpointName, pdDF):
    for idx, row in enumerate(pdDF.values):    
        nLabel, fFeatures = int(row[0]), row[1:]
        payload = csv_serializer.serialize(fFeatures)

        response = runtime_client.invoke_endpoint(
            EndpointName=strEndpointName, 
            ContentType='text/csv',
            Accept='text/csv',
            Body=payload,
            InferenceId=str(idx)
        )
        sleep(0.3)

def invoke_endpoint_forever(strEndpointName, pdDF):
    while True:
        try:
            invoke_endpoint(strEndpointName, pdDF)
        except session.sagemaker_runtime_client.exceptions.ValidationError:
            pass

In [47]:
thread = Thread(target=invoke_endpoint_forever, args=(strEndpointName, pdTest))
thread.start()

### 6.2 View captured data

In [48]:
import json

In [54]:
s3_client = boto3.Session().client("s3")
def get_obj_body(obj_key, strBucketName):
    return s3_client.get_object(Bucket=strBucketName, Key=obj_key).get("Body").read().decode("utf-8")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [72]:
current_endpoint_capture_prefix = os.path.join(
    strPrefix,
    "monitor",
    "data-capture",
    strEndpointName
)
result = s3_client.list_objects(Bucket=strBucketName, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))
print (capture_files[len(capture_files) - 1][:capture_files[len(capture_files) - 1].rfind("/")])

Found Capture Files:
DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/37-45-420-13675920-9041-43dc-ba75-30351ca76e99.jsonl
 DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/40-14-247-5cb76178-c578-4215-8f08-33e9b164eb8f.jsonl
 DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/46-27-464-796254fd-0ab0-4f76-85bd-68362138a32f.jsonl
 DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/47-27-652-9a5038f5-f8ff-403a-a1d6-d72bbfa9995f.jsonl
 DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/48-27-944-3348623f-06ca-4cdf-8e8e-15c54eab1f21.jsonl
 DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/57-13-311-53b1812c-3b87-4c39-8590-34d8feb2db78.jsonl
 DJ-SM-IMD/monitor/data-capture/endpoint-cloud-DJ-SM-IMD-1682062443/AllTraffic/2023/04/21/07/58-13-657-1

In [73]:
capture_file = get_obj_body(capture_files[-1], strBucketName)
print(json.dumps(json.loads(capture_file.split("\n")[1]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "34432.685006666194,47132.685006666194,32.0,55.0,0.0,1.0,750.0,3000.0,85705.0,2019.0,3.0,1.0,1.0,12700.0,10.0,9.0,2.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.2885814309120178,0.0",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "13e6b883-d628-4cd5-92d8-696105e287d7",
    "inferenceId": "968",
    "inferenceTime": "2023-04-21T08:03:14Z"
  },
  "eventVersion": "0"
}


### 6.3 Generate synthetic ground truth

    * Next, start generating ground truth data. The model quality job will fail if there's no ground truth data to merge.
    * "6-1" 과정에서 inference 한 대상에 대한 ground truth (정답)을 `임의로` 생성해 준다


In [61]:
import random
from time import strftime, gmtime
from datetime import datetime
from sagemaker.s3 import S3Uploader

In [62]:
strS3GrondTruthPath=os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "monitor",
    "ground-truth",
    "model",
    strftime("%Y-%m-%d-%H-%M-%S", gmtime())
)
print (f'strS3GrondTruthPath: {strS3GrondTruthPath}')

strS3GrondTruthPath: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/ground-truth/model/2023-04-21-07-57-57


In [63]:
def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    rand = random.random()
    return {
        "groundTruthData": {
            "data": "1" if rand < 0.7 else "0",  # randomly generate positive labels 70% of the time
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }

def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{strS3GrondTruthPath}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [64]:
NUM_GROUND_TRUTH_RECORDS = 1000  # 1000 are the number of rows in data we're sending for inference

fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
upload_ground_truth(fake_records, datetime.utcnow())

Uploading 1000 records to s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/ground-truth/model/2023-04-21-07-57-57/2023/04/21/07/5802.jsonl


### 6.4 Create a monitoring schedule

In [65]:
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor import CronExpressionGenerator

In [66]:
mon_model_schedule_name = "DEMO-model-drift-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

strS3ModelReportPath = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "monitor",
    "report",
    "model"
)

* Create an enpointInput
    - For the monitoring schedule you need to specify how to interpret an endpoint's output. Given that the endpoint in this notebook outputs CSV data, the below code specifies that the `first column of the output, 0, contains a probability` (of churn in this example). You will further specify 0.5 as the cutoff used to determine a positive label (that is, predict that a customer will churn).

In [67]:
endpointInput = EndpointInput(
    endpoint_name=strEndpointName,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

* Create the monitoring schedule to execute every hour.

In [68]:
response = model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_model_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=strS3ModelReportPath,
    problem_type="BinaryClassification",
    ground_truth_input=strS3GrondTruthPath,
    constraints=model_baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: DEMO-model-drift-monitor-schedule-2023-04-21-07-58-44


In [69]:
model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:419974056037:monitoring-schedule/demo-model-drift-monitor-schedule-2023-04-21-07-58-44',
 'MonitoringScheduleName': 'DEMO-model-drift-monitor-schedule-2023-04-21-07-58-44',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2023, 4, 21, 7, 58, 52, 34000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 4, 21, 7, 58, 52, 153000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2023-04-21-07-58-51-491',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'endpoint-cloud-DJ-SM-IMD-1682062443',
 'ResponseMetadata': {'RequestId': '30739e56-dc77-4540-9651-475e37f0b543',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '30739e56-dc77-4540-9651-475e37f0b543',
   'content-type': 'application/x-amz-json-1.1',
   'content-length':

In [100]:
from utils.monitoringjob_utils import run_model_monitor_job_processor

In [101]:
current_endpoint_capture_prefix = os.path.join(
    strPrefix,
    "monitor",
    "data-capture",
    strEndpointName
)
result = s3_client.list_objects(Bucket=strBucketName, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
data_capture_path = capture_files[len(capture_files) - 1][:capture_files[len(capture_files) - 1].rfind("/")]
strS3DataCapturePath = os.path.join(
    "s3://{}".format(strBucketName),
    data_capture_path
)

strS3StatisticsPath = os.path.join(
    strS3BaselineResultsUri,
    "statistics.json"
)
strS3ConstraintsPath = os.path.join(
    strS3BaselineResultsUri,
    "constraints.json"
)
    

NameError: name 'strS3BaselineResultsUri' is not defined

In [ ]:
print (f'data_capture_path: {data_capture_path}')
print (f'strS3BaselineResultsUri: {strS3BaselineResultsUri}')

In [ ]:
processor = run_model_monitor_job_processor(
    strRegionName,
    "ml.m5.xlarge",
    strExecutionRole,
    strS3DataCapturePath,
    strS3StatisticsPath,
    strS3ConstraintsPath,
    strS3ReportPath,
    preprocessor_path=strS3CodePrepUri,
    postprocessor_path=None
)

### 5.1 Constraint suggestion with baseline/training dataset

* copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)

In [110]:
strS3BaselinePrefix = os.path.join(
    strPrefix,
    "monitor",
    "baselining"
)
strS3BaselineDataPrefix = os.path.join(
    strS3BaselinePrefix,
    "data"
)
strS3BaselineResultsPrefix = os.path.join(
    strS3BaselinePrefix,
    "results"
)
strS3BaselineDataUri = os.path.join(
    "s3://{}".format(strBucketName),
    strS3BaselineDataPrefix
)
strS3BaselineResultsUri = os.path.join(
    "s3://{}".format(strBucketName),
    strS3BaselineResultsPrefix
)

print (f'strS3BaselinePrefix: {strS3BaselinePrefix}')
print (f'strS3BaselineDataUri: {strS3BaselineDataUri}')
print (f'strS3BaselineResultsUri: {strS3BaselineResultsUri}')

strS3BaselinePrefix: DJ-SM-IMD/monitor/baselining
strS3BaselineDataUri: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/baselining/data
strS3BaselineResultsUri: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/baselining/results


* change dtype
    - 원하는 형태의 dtype으로 정의 할 수 있음

In [112]:
pdTrain = pd.read_csv(f'{strLocalDataPath}/train.csv')
dicDtypes = {}
for strCol, dtype in zip(pdTrain.columns, pdTrain.dtypes):
    strDtype = str(dtype)
    if strDtype == "int64": dtype = np.float64 
    dicDtypes[strCol] = dtype
dicDtypes
pdTrain = pd.read_csv(f'{strLocalDataPath}/train.csv', dtype=dicDtypes)
pdTrain.dtypes

fraud                                   float64
vehicle_claim                           float64
total_claim_amount                      float64
customer_age                            float64
months_as_customer                      float64
num_claims_past_year                    float64
num_insurers_past_5_years               float64
policy_deductable                       float64
policy_annual_premium                   float64
customer_zip                            float64
auto_year                               float64
num_vehicles_involved                   float64
num_injuries                            float64
num_witnesses                           float64
injury_claim                            float64
incident_month                          float64
incident_day                            float64
incident_dow                            float64
incident_hour                           float64
policy_state_AZ                         float64
policy_state_CA                         

* upload train data to s3

In [113]:
from io import StringIO
s3_client = boto3.client("s3")
s3_key = os.path.join(strS3BaselinePrefix, "data", "train.csv")

with StringIO() as csv_buffer:
    pdTrain.to_csv(csv_buffer, index=False, header=True)
    response = s3_client.put_object(
        Bucket=strBucketName, Key=s3_key, Body=csv_buffer.getvalue()
    )
# training_data_file = open(f'{strLocalDataPath}/train.csv', "rb")
# s3_key = os.path.join(strS3BaselinePrefix, "data", "train.csv")
# boto3.Session().resource("s3").Bucket(strBucketName).Object(s3_key).upload_fileobj(training_data_file)

* Create a baselining job with training dataset

In [114]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

In [115]:
my_default_monitor = DefaultModelMonitor(
    role=strExecutionRole,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=os.path.join(
        strS3BaselineDataUri,
        "train.csv"
    ),
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=strS3BaselineResultsUri,
    wait=True,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2023-04-20-04-34-15-875


.........................2023-04-20 04:38:24,508 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2023-04-20 04:38:25.021952: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-20 04:38:25.021981: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-20 04:38:26.515282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-20 04:38:26.515311: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-20 04:38:26.515335: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be runni

* Explore the generated constraints and statistics

In [116]:
result = s3_client.list_objects(Bucket=strBucketName, Prefix=strS3BaselineResultsPrefix)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
DJ-SM-IMD/monitor/baselining/results/constraints.json
 DJ-SM-IMD/monitor/baselining/results/statistics.json


In [117]:
baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(3)

/tmp/ipykernel_31774/2660869483.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,fraud,Fractional,4000,0,0.032750,1.310000e+02,0.177982,0.0,1.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,vehicle_claim,Fractional,4000,0,17379.745971,6.951898e+07,10122.664885,1000.0,51051.625749,"[{'lower_bound': 1000.0, 'upper_bound': 6005.1...",0.64,2048.0,"[[15405.795272724228, 12735.637099287029, 3288..."
2,total_claim_amount,Fractional,4000,0,41248.395971,1.649936e+08,32595.329899,2100.0,588868.568223,"[{'lower_bound': 2100.0, 'upper_bound': 60776....",0.64,2048.0,"[[19705.79527272423, 30135.637099287032, 25108..."


### 5.2 Analyze collected data for data quality issues

* Upload some test scripts to the S3 bucket for pre- and post-processing

In [118]:
bucket = boto3.Session().resource("s3").Bucket(strBucketName)
strLocalCodePrefix = os.path.join(os.getcwd(), "source", "monitor")
strS3CodePrepKey = os.path.join(
    strPrefix,
    "monitor",
    "code",
    "prep",
    "preprocessor.py"
)
strS3CodePrepUri = os.path.join(
    "s3://{}".format(strBucketName),
    strS3CodePrepKey
)
strS3CodePostpKey = os.path.join(
    strPrefix,
    "monitor",
    "code",
    "postp",
    "postprocessor.py"
)
strS3CodePostpUri = os.path.join(
    "s3://{}".format(strBucketName),
    strS3CodePostpKey
)
print (strLocalCodePrefix)
print (strS3CodePostpUri)
print (strS3CodePrepUri)
boto3.Session().resource("s3").Bucket(strBucketName).Object(strS3CodePostpKey).upload_file(strLocalCodePrefix + "/postprocessor.py")
boto3.Session().resource("s3").Bucket(strBucketName).Object(strS3CodePrepKey).upload_file(strLocalCodePrefix + "/preprocessor.py")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


/home/ec2-user/SageMaker/sagemaker-immersion-day/source/monitor
s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/code/postp/postprocessor.py
s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/monitor/code/prep/preprocessor.py


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


* Create a schedule

In [119]:
from time import strftime, gmtime
from sagemaker.model_monitor import CronExpressionGenerator

In [120]:
mon_schedule_name = "DEMO-data-drift-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

strS3ReportPath = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "monitor",
    "report"
)

mon_schedule_name

'DEMO-data-drift-monitor-schedule-2023-04-20-04-41-26'

In [121]:
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=strEndpointName,
    record_preprocessor_script=strS3CodePrepUri,
    # post_analytics_processor_script=strS3CodePostpUri,
    output_s3_uri=strS3ReportPath,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: DEMO-data-drift-monitor-schedule-2023-04-20-04-41-26


### 5.3 Violations report
- https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-monitor-interpreting-violations.html

    - data_type_check
        - 현재 실행의 데이터 유형이 기준 데이터 세트의 데이터 유형과 다를 경우 이 위반에 플래그가 지정됩니다.
        - 기준 단계 동안 생성된 제약 조건은 각 열에 대해 추론된 데이터 유형을 제안합니다. 위반으로 플래그가 지정된 경우 임계값을 조정하도록 monitoring_config.datatype_check_threshold 파라미터를 튜닝할 수 있습니다.

    - completeness_check	
        - 현재 실행에서 관찰된 완전성(null이 아닌 항목의 %)이 기능별로 지정된 완전성 임계값에 지정된 임계값을 초과하면 이 위반에 플래그가 지정됩니다.
        - 기준 단계 동안 생성된 제약 조건은 완전성 값을 제안합니다.

    - baseline_drift_check	
        - 현재 데이터 세트와 기준 데이터 세트 간에 계산된 분포 거리가 monitoring_config.comparison_threshold에 지정된 임계값보다 크면 이 위반에 플래그가 지정됩니다.
    
    - missing_column_check	
        - 현재 데이터 세트의 열 수가 기준 데이터 세트의 개수보다 작으면 이 위반에 플래그가 지정됩니다.

    - extra_column_check	
        - 현재 데이터 세트의 열 수가 기준의 개수보다 많으면 이 위반에 플래그가 지정됩니다.

    - categorical_values_check	
        - 현재 데이터 세트의 알 수 없는 값이 기준 데이터 세트보다 더 많으면 이 위반에 플래그가 지정됩니다. 이 값은 monitoring_config.domain_content_threshold의 임계값에 의해 결정됩니다.

In [128]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option("display.max_colwidth", None)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

/tmp/ipykernel_31774/4018518515.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])


""
